In [ ]:
#REAL PROGRAM
# import the necessary packages
from sklearn.cluster import DBSCAN
from __future__ import print_function
from imutils.object_detection import non_max_suppression
from imutils import paths
import numpy as np
import imutils
import cv2
import sys

# initialize the HOG descriptor/person detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# loop over the image paths
caps = cv2.VideoCapture(r'C:/Users/Emyr/Documents/Jupyter/pedestrian-detection/video/Pedestrians.mp4')
ret, frame = caps.read()

count = int(caps.get(cv2.CAP_PROP_FRAME_COUNT))

while(count > 1):
    ret, frame = caps.read()
    frame = imutils.resize(frame, width = min(frame.shape[0], frame.shape[1]))
    orig = frame.copy()

    # detect people in the image
    (rects, weights) = hog.detectMultiScale(frame, winStride=(2, 2), padding=(8, 8), scale=0.5) #scale-1.05#

    if len(rects) != 0:
        #print(len(rects))
        center = []
    
    # draw the original bounding boxes
    for (x, y, w, h) in rects:
        cv2.rectangle(orig, (x, y), (x + w, y + h), (0, 0, 255), 2)

    # apply non-maxima suppression to the bounding boxes using a
    # fairly large overlap threshold to try to maintain overlapping
    # boxes that are still people
    rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in rects])
    pick = non_max_suppression(rects, probs=None, overlapThresh=0.65)

    # draw the final bounding boxes
    for (xA, yA, xB, yB) in pick:
      
        cv2.rectangle(frame, (xA, yA), (xB, yB), (0, 255, 0), 2)
        #print(xA, yA, xB, yB)
        
        CentxPos = int((xA + xB)/2)
        CentyPos = int((yA + yB)/2)
    
        cv2.circle(frame,(CentxPos, CentyPos), 5, (0,255,0), -1)
        center.append([CentxPos, CentyPos])

        clustering = DBSCAN(eps=3, min_samples=2).fit(center)
            
        
    cv2.imshow("After NMS", frame)
    cv2.waitKey(1) 
    count = count - 1

caps.release()
cv2.destroyAllWindows()

In [ ]:
#TEST IMAGE
# import the necessary packages
from sklearn.cluster import DBSCAN
from __future__ import print_function
from imutils.object_detection import non_max_suppression
from imutils import paths
import numpy as np
import imutils
import cv2

# initialize the HOG descriptor/person detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# loop over the image paths
imagePaths = list(paths.list_images(r"C:\Users\Emyr\Documents\Jupyter\pedestrian-detection\images"))

for imagePath in imagePaths:
    # load the image and resize it to (1) reduce detection time
    # and (2) improve detection accuracy
    image = cv2.imread(imagePath)
    print (image.shape[0])
    print (image.shape[1])
    print (image.shape[2])
    image = imutils.resize(image, width=min(6400, image.shape[1]))
    orig = image.copy()

    # detect people in the image
    (rects, weights) = hog.detectMultiScale(image, winStride=(4, 4), padding=(8, 8), scale=1.05)

    # draw the original bounding boxes
    #for (x, y, w, h) in rects:
    #    cv2.rectangle(orig, (x, y), (x + w, y + h), (0, 0, 255), 2)

    # apply non-maxima suppression to the bounding boxes using a
    # fairly large overlap threshold to try to maintain overlapping
    # boxes that are still people
    #rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in rects])
    pick = non_max_suppression(rects, probs=None, overlapThresh=0.65)

    # draw the final bounding boxes
    #for (xA, yA, xB, yB) in pick:
    cv2.rectangle(image, (5, 5), (100, 500), (0, 255, 0), 2)
    
    CentxPos = int((5 +100)/2)
    CentyPos = int((5 + 500)/2)
    
    cv2.circle(image,(CentxPos, CentyPos), 5, (0,255,0), -1)
    # show some information on the number of bounding boxes
    filename = imagePath[imagePath.rfind("/") + 1:]
    print("[INFO] {}: {} original boxes, {} after suppression".format(filename, len(rects), len(pick)))

    # show the output images
    #cv2.imshow("Before NMS", orig)
    cv2.imshow("After NMS", image)
    cv2.waitKey(0)


In [ ]:
#TEST PROGRAM - Alternate Clustering
# import the necessary packages
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AffinityPropagation
import matplotlib.pyplot as plt
from __future__ import print_function
from imutils.object_detection import non_max_suppression
from imutils import paths
import numpy as np
import imutils
import cv2
import sys
import matplotlib.pyplot as plt
from itertools import cycle
%matplotlib inline 

# initialize the HOG descriptor/person detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# loop over the image paths
caps = cv2.VideoCapture(r'C:/Users/Emyr/Documents/Jupyter/pedestrian-detection/video/Ped2.MOV')
ret, frame = caps.read()

count = int(caps.get(cv2.CAP_PROP_FRAME_COUNT))

while(count > 1):
    
    center = []
    Detection = False
    ret, frame = caps.read()
    frame = imutils.resize(frame, width = min(frame.shape[0], frame.shape[1]))
    orig = frame.copy()

    # detect people in the image
    (rects, weights) = hog.detectMultiScale(frame, winStride=(2, 2), padding=(8, 8), scale=0.5) #scale-1.05#
    
    # draw the original bounding boxes
    for (x, y, w, h) in rects:
        cv2.rectangle(orig, (x, y), (x + w, y + h), (0, 0, 255), 2)

    # apply non-maxima suppression to the bounding boxes using a
    # fairly large overlap threshold to try to maintain overlapping
    # boxes that are still people
    rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in rects])
    pick = non_max_suppression(rects, probs=None, overlapThresh=0.65)

    # draw the final bounding boxes
    count2  = 0
    
    for (xA, yA, xB, yB) in pick:
      
        cv2.rectangle(frame, (xA, yA), (xB, yB), (0, 255, 0), 2)
        #print(xA, yA, xB, yB)
        
        CentxPos = int((xA + xB)/2)
        CentyPos = int((yA + yB)/2)
    
        cv2.circle(frame,(CentxPos, CentyPos), 5, (0,255,0), -1)
        count2 += 1
        
        if len(rects) >1:
               center.append([CentxPos, CentyPos])
        
        
    if count2 >= 2:
        Detection = True
        
             
    if Detection == True:  
        
        center = StandardScaler().fit_transform(center)
        af = AffinityPropagation().fit(center)
        
        cluster_centers_indices = af.cluster_centers_indices_
        labels = af.labels_

        n_clusters_ = len(cluster_centers_indices)
        
        plt.close('all')
        plt.figure(1)
        plt.clf()

        colors = cycle('bgrcmykbgrcmykbgrcmykbgrcmyk')
        
        for k, col in zip(range(n_clusters_), colors):
           
           # fig, ax = plt.subplots()
           # ax.set_xlim(0,frame.shape[1]) #405.frame.shape[0]
           # ax.set_ylim(frame.shape[0], 0) #720
        
            class_members = labels == k
            center = np.asarray(center)
            cluster_center = center[cluster_centers_indices[k]]
            
            #print(type(inp1))

            plt.plot(center[class_members, 0],center[class_members, 1], col + '.')
            plt.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col, markeredgecolor='k', markersize=14)
              
            for x in center[class_members]:
                plt.plot([cluster_center[0], x[0]], [cluster_center[1], x[1]], col)

        plt.title('Estimated number of clusters: %d' % n_clusters_)
        plt.show()
        
        cv2.imshow("After NMS", frame)
        cv2.waitKey(0) 
        count = count - 1

    else:
        
        cv2.imshow("After NMS", frame)
        cv2.waitKey(1) 
        count = count - 1

caps.release()
cv2.destroyAllWindows()

In [ ]:
#TEST PROGRAM - CUTTING DOWN
# import the necessary packages
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from __future__ import print_function
from imutils.object_detection import non_max_suppression
from imutils import paths
import numpy as np
import imutils
import cv2
import sys
import matplotlib.pyplot as plt
from itertools import cycle
%matplotlib inline  

# initialize the HOG descriptor/person detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

# loop over the image paths
caps = cv2.VideoCapture(r'C:/Users/Emyr/Documents/Jupyter/pedestrian-detection/video/Ped2.MOV')
ret, frame = caps.read()

count = int(caps.get(cv2.CAP_PROP_FRAME_COUNT))

while(count > 1):
    
    center = []
    Detection = False
    ret, frame = caps.read()
    frame = imutils.resize(frame, width = min(frame.shape[0], frame.shape[1]))
    orig = frame.copy()

    # detect people in the image
    (rects, weights) = hog.detectMultiScale(frame, winStride=(1,1), padding=(0, 0), scale=1)
    
    # draw the original bounding boxes
    for (x, y, w, h) in rects:
        cv2.rectangle(orig, (x, y), (x + w, y + h), (0, 0, 255), 2)

    # apply non-maxima suppression to the bounding boxes using a
    # fairly large overlap threshold to try to maintain overlapping
    # boxes that are still people
    rects = np.array([[x, y, x + w, y + h] for (x, y, w, h) in rects])
    pick = non_max_suppression(rects, probs=None, overlapThresh=0.65)

    # draw the final bounding boxes
    count2  = 0
    
    for (xA, yA, xB, yB) in pick:
      
        cv2.rectangle(frame, (xA, yA), (xB, yB), (0, 255, 0), 2)
        #print(xA, yA, xB, yB)
        
        CentxPos = int((xA + xB)/2)
        CentyPos = int((yA + yB)/2)
    
        cv2.circle(frame,(CentxPos, CentyPos), 5, (0,255,0), -1)
        count2 += 1
        
        if len(rects) >1:
               center.append([CentxPos, CentyPos])
        
        
    if count2 >= 2:
        Detection = True
        
             
    if Detection == True:    
        
        db = DBSCAN(eps= 85, min_samples = 3).fit(center)
        #core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
        #core_samples_mask[db.core_sample_indices_] = True
        labels = db.labels_
        
        # Number of clusters in labels, ignoring noise if present.
        n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
        n_noise_ = list(labels).count(-1)

        # Black removed and is used for noise instead.
        unique_labels = set(labels)
        colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]
        center = np.asarray(center)
        
        fig, ax = plt.subplots()
            
        ax.set_xlim(0,frame.shape[1])
        ax.set_ylim(frame.shape[0], 0)
        
        for k, col in zip(unique_labels, colors):
            
            if k == -1:
                # Black used for noise.
                col = [0, 0, 0, 1]

            class_member_mask = (labels == k)
            
            xy = center[class_member_mask]
            plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col), markeredgecolor='k', markersize=8)
            
        plt.title('Estimated number of clusters: %d' % n_clusters_)
        plt.show()   
        cv2.imshow("After NMS", frame)

        cv2.waitKey(1) 
        count = count - 1

    else:
        
        cv2.imshow("After NMS", frame)
        cv2.waitKey(1) 
        count = count - 1
    
        
caps.release()
cv2.destroyAllWindows()